In [12]:
using Turing, Distributions
using PyPlot, PyCall
@pyimport matplotlib.cm as cm
@pyimport numpy as np

In [13]:
# Load transition matrix T and observations obs
include(Pkg.dir("Turing")*"/notebooks/data/hmmdemo.data.jl");

In [28]:
K = 10
N = 51
initial = fill(1.0 / K, K)
means = collect(1.0:K)

@model hmmdemo begin
    states = tzeros(Int,N)
    # T = TArray{Array{Float64,}}
    
    # Prior over T
    # for i=1:K, @assume T[i,:] ~ Dirichlet(ones(K)./K); end
    
    @assume states[1] ~ Categorical(initial)
    for i = 2:N
        @assume states[i] ~ Categorical(vec(T[states[i-1],:]))
        @observe obs[i] ~ Normal(means[states[i]], 2)
    end
    @predict states
end

hmmdemo (generic function with 1 method)

In [33]:
chain = sample(hmmdemo, PG(100,1000));

In [36]:
# Print the 1st sample trajectory
chain.value[1].value[:states]'
chain.value[1000]

Turing.Sample(0.001,Dict{Symbol,Any}(:states=>[5,8,9,8,9,1,5,8,9,8  …  9,8,9,4,5,8,9,8,9,7]))

In [37]:
# plot in an external window since it doesn't work yet in IJulia
pygui(true)
# tell PyPlot that the plot is interactive
PyPlot.ion()
# . . . and that previous plots are overwritten
PyPlot.hold(false)
colors = cm.rainbow(np.linspace(0, 1, K))
fig = gcf()
fig[:figsize]=(8,6)

for i=1:length(chain.value)
    # Plot trajectories
    plot(1:51, means[chain.value[i].value[:states]], 1:51, obs, "rs")
    
    # Then force the draw
    pause(0.05)
end